# Fetch List of Cases

In [1]:
import requests
import pandas as pd
import numpy as np
import re
import pickle

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm

In [2]:
data = pd.DataFrame()

In [4]:
for i in tqdm(range(49,0,-1)):
    data_page = []
    # set the URL
    url = 'http://sipp.pn-jakartapusat.go.id/http://sipp.pn-jakartapusat.go.id/list_perkara/page/'+str(i)+'/OXB2Nkc3U0lzUVoza1pGcjMrNUx6QkZyRU5ZU1cxekczRnYrcGY0ZDN3elVpKzFjeDJjWXAzOWRTOU9ENy81ejFFUlFSRzkxWURhaWVzNWVKYUdYTGc9PQ==/key/col/2:147'
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(20)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    # parsing data row by row
    table_rows = soup.find_all('tr')
    for row in table_rows:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') if ele.get_text() != '[detil]' else ele.find('a')['href']  for ele in cols]
        data_page.append(cols)
    
    page_table = pd.DataFrame(data_page[1:-1],columns=data_page[0]).set_index('Nomor Perkara').drop('No',axis=1)
    data = data.append(page_table)

A Jupyter Widget

In [12]:
# Remove duplicates by case ID
data = data[~data.index.duplicated(keep='first')]

In [26]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'wb') as f:
    pickle.dump(data, f)

# Crawled data to CSV
data.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.csv')

# Fetch Case Details

## Fetch Judge and Clerk Name

In [15]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [16]:
case_list = list(datax.index.unique())
len(case_list)

964

In [5]:
data_hakim = pd.DataFrame()
data_panitera = pd.DataFrame()
data_sidang_pertama = pd.DataFrame()

In [6]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
    url_hakim = url.replace("show_detil","show_penetapan")
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url_hakim)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()
    
    # Tabel Hakim
    kolom_tabel_hakim = ['Nomor Perkara','Tanggal Penetapan', 'Nama Hakim/Majelis Hakim', 'Posisi', 'Aktif']
    hakim_kasus = []
    table_hakim = soup.find_all('div',class_='cssTable')[0].find_all('tr')
    for row in table_hakim:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        hakim_kasus.append(cols)
    
    page_table = pd.DataFrame(hakim_kasus[1:],columns=kolom_tabel_hakim).set_index('Nomor Perkara')
    data_hakim = data_hakim.append(page_table)
    
    # Tabel Panitera
    kolom_tabel_panitera = ['Nomor Perkara','Tanggal Penetapan', 'Nama Panitera Pengganti', 'Aktif']
    panitera_kasus = []
    table_panitera = soup.find_all('div',class_='cssTable')[1].find_all('tr')
    for row in table_panitera:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        panitera_kasus.append(cols)
    
    table_panitera = pd.DataFrame(panitera_kasus[1:],columns=kolom_tabel_panitera).set_index('Nomor Perkara')
    data_panitera = data_panitera.append(table_panitera)
    
    # Tabel Hari Sidang Pertama
    kolom_tabel_pertama = ['Nomor Perkara','Tanggal Penetapan', 'Tanggal Sidang Pertama']
    sidang_pertama_kasus = []
    table_sidang_pertama = soup.find_all('div',class_='cssTable')[2].find_all('tr')
    for row in table_sidang_pertama:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        sidang_pertama_kasus.append(cols)
    
    table_sidang_pertama = pd.DataFrame(sidang_pertama_kasus[1:],columns=kolom_tabel_pertama).set_index('Nomor Perkara')
    data_sidang_pertama = data_sidang_pertama.append(table_sidang_pertama)    

A Jupyter Widget

In [9]:
data_hakim.head()

,Tanggal Penetapan,Nama Hakim/Majelis Hakim,Posisi,Aktif
Nomor Perkara,,,,
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Ketua,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya


In [11]:
data_panitera.tail()

,Tanggal Penetapan,Nama Panitera Pengganti,Aktif
Nomor Perkara,,,
00012/PID.B/TPKOR/2011/PN.JKT.PST,"Senin, 21 Mar. 2011","WIDI ASTUTI, SH",Ya
00013/PID.B/TPKOR/2011/PN.JKT.PST,"Senin, 28 Mar. 2011","TEUKU UMAR, SH. MH.",Ya
00014/PID.B/TPKOR/2011/PN.JKT.PST,"Senin, 28 Mar. 2011","RUSTIANI, SH",Ya
00015/PID.B/TPKOR/2011/PN.JKT.PST,"Rabu, 30 Mar. 2011","FATONI, SH",Ya
00015/PID.B/TPKOR/2011/PN.JKT.PST,"Rabu, 30 Mar. 2011",WIJI ASTUTI,Ya


In [12]:
data_sidang_pertama.tail()

,Tanggal Penetapan,Tanggal Sidang Pertama
Nomor Perkara,,
00011/PID.B/TPKOR/2011/PN.JKT.PST,"Selasa, 22 Mar. 2011","Senin, 04 Apr. 2011"
00012/PID.B/TPKOR/2011/PN.JKT.PST,"Kamis, 24 Mar. 2011","Selasa, 29 Mar. 2011"
00013/PID.B/TPKOR/2011/PN.JKT.PST,"Senin, 28 Mar. 2011","Selasa, 29 Mar. 2011"
00014/PID.B/TPKOR/2011/PN.JKT.PST,"Rabu, 30 Mar. 2011","Senin, 11 Apr. 2011"
00015/PID.B/TPKOR/2011/PN.JKT.PST,"Jumat, 01 Apr. 2011","Kamis, 07 Apr. 2011"


In [13]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.pickle', 'wb') as f:
    pickle.dump(data_hakim, f)
    
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_clerk_list.pickle', 'wb') as f:
    pickle.dump(data_panitera, f)
    
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_first_court_day.pickle', 'wb') as f:
    pickle.dump(data_sidang_pertama, f)

# Crawled data to CSV
data_hakim.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.csv')
data_panitera.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_clerk_list.csv')
data_sidang_pertama.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_first_court_day_list.csv')

## Fetch General Info of Case

In [7]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [8]:
case_list = list(datax.index.unique())
len(case_list)

964

In [11]:
general_info_table = pd.DataFrame()

In [12]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
 
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()    
      
    # Info Umum
    general_info = soup.find_all('table', id='infoPerkara')[0].find_all('tr')
    
    # Skip the table-inside-table situation
    test = []
    for i in general_info:
        if len(i.find_parents("tr")) == 0:
            if i.div is None:
                test.append(i)
    kolom_general_info = ['Nomor Perkara','Atribut','Nilai']
    
    info = []
    for row in test:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        info.append(cols)

    page_info = pd.DataFrame(info,columns=kolom_general_info).set_index('Nomor Perkara')
    general_info_table = general_info_table.append(page_info)

A Jupyter Widget

In [14]:
len(list(general_info_table.index.unique()))

964

In [74]:
# Transpose it!
df = general_info_table
df.reset_index(inplace=False)
transposed_df = df.pivot(columns='Atribut',values='Nilai')
transposed_df

Atribut,Dakwaan,Klasifikasi Perkara,Nomor Perkara,Nomor Surat Pelimpahan,Penasihat Hukum Terdakwa,Pihak Dipublikasikan,Tanggal Pendaftaran,Tanggal Surat Pelimpahan
Nomor Perkara,,,,,,,,
00001/PID.B/TPKOR/2011/PN.JKT.PST,PASAL : 21 UU R.I No.31/ 1999 JO UU R.I No.20/...,Tindak Pidana Korupsi,00001/PID.B/TPKOR/2011/PN.JKT.PST,,\n,Ya,"Selasa, 18 Jan. 2011",-
00001/PID.B/TPKOR/2012/PN.JKT.PST,KESATU : Pasal 2 (1) jo. Pasal 18 (1) b UU No....,Tindak Pidana Korupsi,00001/PID.B/TPKOR/2012/PN.JKT.PST,,\n,Ya,"Rabu, 04 Jan. 2012",-
00002/PID.B/TPKOR/2011/PN.JKT.PST,PASAL : 2 AYAT (1) JO PASAL 18 UU R.I No.31/ T...,Tindak Pidana Korupsi,00002/PID.B/TPKOR/2011/PN.JKT.PST,,\n,Ya,"Rabu, 26 Jan. 2011",-
00002/PID.B/TPKOR/2012/PN.JKT.PST,KESATU : Pasal 2 (1) jo. Pasal 18 (1) huruf b ...,Tindak Pidana Korupsi,00002/PID.B/TPKOR/2012/PN.JKT.PST,,\n,Ya,"Rabu, 04 Jan. 2012",-
00003/PID.B/TPKOR/2011/PN.JKT.PST,PASAL : 2 AYAT (1) JO PASAL 18 UU No.31 TAHUN ...,Tindak Pidana Korupsi,00003/PID.B/TPKOR/2011/PN.JKT.PST,,\n,Ya,"Rabu, 09 Feb. 2011",-
00003/PID.B/TPKOR/2012/PN.JKT.PST,KESATU : Pasal 2 (1) jo. Pasal 18 UU Nomor 31/...,Tindak Pidana Korupsi,00003/PID.B/TPKOR/2012/PN.JKT.PST,,\n,Ya,"Selasa, 10 Jan. 2012",-
00004/PID.B/TPKOR/2011/PN.JKT.PST,PASAL : 2 (1) JO PASAL 18 (1)B UU R.I No.31/ 1...,Tindak Pidana Korupsi,00004/PID.B/TPKOR/2011/PN.JKT.PST,,\n,Ya,"Rabu, 16 Feb. 2011",-
00004/PID.B/TPKOR/2012/PN.JKT.PST,PERTAMA : Pasal 3 jo. Pasal 18 UU Nomor 31/199...,Tindak Pidana Korupsi,00004/PID.B/TPKOR/2012/PN.JKT.PST,,\n,Ya,"Kamis, 19 Jan. 2012",-
00005/PID.B/TPKOR/2011/PN.JKT.PST,PASAL : 15 JO PASAL 5 AYAT (1)A UU R.I No.31/...,Tindak Pidana Korupsi,00005/PID.B/TPKOR/2011/PN.JKT.PST,,\n,Ya,"Rabu, 16 Feb. 2011",-


In [75]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/general_info_table.pickle', 'wb') as f:
    pickle.dump(transposed_df, f)

transposed_df.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/general_info_table.csv')